In [ ]:
#Credits and Thanks
#https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/

In [ ]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession
from random import randint

In [4]:
spark=SparkSession.builder.appName('Join').getOrCreate()

In [39]:
names = ['James','John','Robert','Michael','David','Mary','Patricia','Jennifer','Linda','Elizabeth']
emp = []
empColumns = ['id','name','age','emp_dept_id']
dept = [("Finance",1),("Marketing",2),("Sales",3),("IT",4),("HR",5)]
deptColumns = ["dept_name","dept_id"]
for i in range(0,2000):
  emp.append([i,names[randint(0,9)],randint(20,60),randint(1,5)])
df_emp = spark.createDataFrame(data=emp, schema = empColumns)
df_dept = spark.createDataFrame(data=dept, schema=deptColumns)

In [40]:
df_emp.printSchema(), df_dept.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- emp_dept_id: long (nullable = true)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)



(None, None)

In [41]:
df_emp.show()

+---+---------+---+-----------+
| id|     name|age|emp_dept_id|
+---+---------+---+-----------+
|  0|    David| 33|          1|
|  1|Elizabeth| 20|          5|
|  2|    James| 46|          4|
|  3| Jennifer| 59|          1|
|  4|  Michael| 57|          4|
|  5|   Robert| 29|          1|
|  6| Jennifer| 54|          2|
|  7|Elizabeth| 57|          2|
|  8|  Michael| 42|          5|
|  9|  Michael| 46|          3|
| 10| Jennifer| 41|          4|
| 11|   Robert| 25|          4|
| 12|     Mary| 37|          2|
| 13| Patricia| 60|          2|
| 14|   Robert| 56|          1|
| 15|     Mary| 39|          4|
| 16|  Michael| 56|          5|
| 17|     John| 55|          4|
| 18|    James| 51|          5|
| 19|     John| 32|          5|
+---+---------+---+-----------+
only showing top 20 rows



In [42]:
df_dept.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|      1|
|Marketing|      2|
|    Sales|      3|
|       IT|      4|
|       HR|      5|
+---------+-------+



In [43]:
df_join = df_emp.join(df_dept,df_emp.emp_dept_id ==  df_dept.dept_id,"inner")
df_join.show()

+---+---------+---+-----------+---------+-------+
| id|     name|age|emp_dept_id|dept_name|dept_id|
+---+---------+---+-----------+---------+-------+
|  0|    David| 33|          1|  Finance|      1|
|  3| Jennifer| 59|          1|  Finance|      1|
|  5|   Robert| 29|          1|  Finance|      1|
| 14|   Robert| 56|          1|  Finance|      1|
| 31|    Linda| 45|          1|  Finance|      1|
| 32|   Robert| 26|          1|  Finance|      1|
| 36|     John| 41|          1|  Finance|      1|
| 37|    James| 51|          1|  Finance|      1|
| 41|    David| 36|          1|  Finance|      1|
| 46|    Linda| 59|          1|  Finance|      1|
| 62|    James| 54|          1|  Finance|      1|
| 68|  Michael| 35|          1|  Finance|      1|
| 72| Jennifer| 40|          1|  Finance|      1|
| 75|    Linda| 24|          1|  Finance|      1|
| 80|Elizabeth| 29|          1|  Finance|      1|
| 82|    Linda| 32|          1|  Finance|      1|
| 91| Jennifer| 34|          1|  Finance|      1|


In [44]:
df_join.groupby("dept_name").count().show()

+---------+-----+
|dept_name|count|
+---------+-----+
|    Sales|  383|
|       HR|  396|
|  Finance|  392|
|Marketing|  399|
|       IT|  430|
+---------+-----+



In [45]:
df_join.filter('id > 70').groupby("dept_name").count().show()

+---------+-----+
|dept_name|count|
+---------+-----+
|    Sales|  375|
|       HR|  377|
|  Finance|  380|
|Marketing|  378|
|       IT|  419|
+---------+-----+



In [46]:
df_join.filter('age > 40 or dept_id = 5').select('name','age','dept_id','dept_name').sort("age",ascending=False).show()

+---------+---+-------+---------+
|     name|age|dept_id|dept_name|
+---------+---+-------+---------+
|   Robert| 60|      2|Marketing|
|Elizabeth| 60|      4|       IT|
|Elizabeth| 60|      2|Marketing|
|    James| 60|      1|  Finance|
|     John| 60|      2|Marketing|
|    James| 60|      1|  Finance|
| Patricia| 60|      3|    Sales|
|   Robert| 60|      1|  Finance|
|    James| 60|      3|    Sales|
|    David| 60|      2|Marketing|
|   Robert| 60|      3|    Sales|
|Elizabeth| 60|      2|Marketing|
|     John| 60|      3|    Sales|
|    David| 60|      2|Marketing|
|Elizabeth| 60|      3|    Sales|
|   Robert| 60|      2|Marketing|
|   Robert| 60|      3|    Sales|
| Patricia| 60|      3|    Sales|
| Jennifer| 60|      4|       IT|
|    David| 60|      3|    Sales|
+---------+---+-------+---------+
only showing top 20 rows



In [47]:
df_join.filter('dept_name = "Finance"').agg({'age':'avg'}).show()

+-----------------+
|         avg(age)|
+-----------------+
|40.02040816326531|
+-----------------+



In [48]:
df_join.groupby('dept_name').avg('age').sort('avg(age)',ascending=False).show()

+---------+------------------+
|dept_name|          avg(age)|
+---------+------------------+
|    Sales| 40.74934725848564|
|Marketing|40.711779448621556|
|  Finance| 40.02040816326531|
|       IT| 39.53488372093023|
|       HR|  39.2979797979798|
+---------+------------------+

